# 0.pre-processing to batch download attachments from ArcGIS file geodatabase

GCP data must be exported from ArcGIS Online to the format ArcGIS File Geodatabase. Run the following script inside ArcMap to parse attached images from binary to .jpg. 

Reference: https://support.esri.com/en/technical-article/000011912

In [ ]:
import arcpy
from arcpy import da
import os

inTable = arcpy.GetParameterAsText(0)
fileLocation = arcpy.GetParameterAsText(1)

with da.SearchCursor(inTable, ['DATA', 'ATT_NAME', 'ATTACHMENTID']) as cursor:
    for item in cursor:
        attachment = item[0]
        filenum = "ATT" + str(item[2]) + "_"
        filename = filenum + str(item[1])
        open(fileLocation + os.sep + filename, 'wb').write(attachment.tobytes())
        del item
        del filenum
        del filename
        del attachment

# 1.convert decimal degree to degree-minute-second

In [1]:
import csv
import pyproj
import numpy as np
import pandas as pd
import geopandas as gpd
import datetime as dt

In [4]:
location_gcp_decimal = pd.read_csv("location_gcp__decimal.csv")
elevation_gcp_decimal = pd.read_csv("elevation_gcp__decimal.csv")
location_gcp_decimal.columns = ['lat', 'lon', 'elipsoidal_h', 'date', 'img_ix']
elevation_gcp_decimal.columns = ['lat', 'lon', 'elipsoidal_h', 'date']

In [5]:
location_gcp_decimal[['lat_degree', 'lat_minute', 'lat_second', 'lon_degree', 'lon_minute', 'lon_second', 'ix']] = pd.DataFrame([[np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, 'POINT_']], index=location_gcp_decimal.index)
elevation_gcp_decimal[['lat_degree', 'lat_minute', 'lat_second', 'lon_degree', 'lon_minute', 'lon_second', 'ix']] = pd.DataFrame([[np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, 'POINT_']], index=elevation_gcp_decimal.index)

In [6]:
location_gcp_decimal['ix'] = location_gcp_decimal.index
location_gcp_decimal['ix'] = location_gcp_decimal['ix'].astype(str)
location_gcp_decimal['ix'] = location_gcp_decimal['ix'] + '_POINT'

elevation_gcp_decimal['ix'] = elevation_gcp_decimal.index
elevation_gcp_decimal['ix'] = elevation_gcp_decimal['ix'].astype(str)
elevation_gcp_decimal['ix'] = elevation_gcp_decimal['ix'] + '_POINT'

location_gcp_decimal

,lat,lon,elipsoidal_h,date,img_ix,lat_degree,lat_minute,lat_second,lon_degree,lon_minute,lon_second,ix
0,40.472757,-74.339784,-26.638,2021-06-07,1,NaN,NaN,NaN,NaN,NaN,NaN,0_POINT
1,40.472669,-74.339782,-26.319,2021-06-07,2,NaN,NaN,NaN,NaN,NaN,NaN,1_POINT
2,40.472780,-74.339924,-26.892,2021-06-07,3,NaN,NaN,NaN,NaN,NaN,NaN,2_POINT
3,40.472836,-74.339937,-27.075,2021-06-07,4,NaN,NaN,NaN,NaN,NaN,NaN,3_POINT
4,40.473462,-74.339932,-28.139,2021-06-07,5,NaN,NaN,NaN,NaN,NaN,NaN,4_POINT
...,...,...,...,...,...,...,...,...,...,...,...,...
384,40.595668,-74.448605,-9.705,2021-06-18,433,NaN,NaN,NaN,NaN,NaN,NaN,384_POINT
385,40.597088,-74.460974,-13.980,2021-06-18,434,NaN,NaN,NaN,NaN,NaN,NaN,385_POINT
386,40.603906,-74.470785,-14.754,2021-06-18,435,NaN,NaN,NaN,NaN,NaN,NaN,386_POINT
387,40.603906,-74.470785,-14.754,2021-06-18,436,NaN,NaN,NaN,NaN,NaN,NaN,387_POINT


In [9]:
def decimal_to_dms(df):
    
    lat = df['lat'].abs()
    lon = df['lon'].abs()
    
    degree_lat = lat.astype(int)
    minute_lat = ((lat - degree_lat) * 60).astype(int)
    second_lat = (((lat - degree_lat - minute_lat / 60)) * 3600).round(2)
    
    degree_lon = lon.astype(int)
    minute_lon = ((lon - degree_lon) * 60).astype(int)
    second_lon = (((lon - degree_lon - minute_lon / 60)) * 3600).round(2)
    
    df['lat_degree'] = degree_lat
    df['lat_minute'] = minute_lat
    df['lat_second'] = second_lat
    
    df['lon_degree'] = degree_lon
    df['lon_minute'] = minute_lon
    df['lon_second'] = second_lon
    
    temp = pd.concat([degree_lat, minute_lat, second_lat, degree_lon, minute_lon, second_lon, pd.Series(' ' * 8, index=df.index), df['ix']], axis=1)
    temp.columns = ['lat_degree', 'lat_minute', 'lat_second', 'lon_degree', 'lon_minute', 'lon_second', 'spaces', 'ix']
    return(temp)

In [10]:
location_for_geoid = decimal_to_dms(location_gcp_decimal)
location_for_geoid.to_csv("location_for_geoid.csv", sep=' ', index=False, header=False, quoting=csv.QUOTE_NONE, quotechar="",  escapechar=" ")

elevation_for_geoid = decimal_to_dms(elevation_gcp_decimal)
elevation_for_geoid.to_csv("elevation_for_geoid.csv", sep=' ', index=False, header=False, quoting=csv.QUOTE_NONE, quotechar="",  escapechar=" ")

# 2. batch to get undulation at

https://geodesy.noaa.gov/GEOID/GEOID18/computation.html

# 3. compute orthoheight

In [12]:
location_undulation = pd.read_csv("location_undulation.csv", names=['lat_degree',
                                                        'lat_minute',
                                                        'lat_second',
                                                        'lon_degree',
                                                        'lon_minute',
                                                        'lon_second',
                                                        'geoid',
                                                        'error',
                                                        'ix'], delim_whitespace=True)

elevation_undulation = pd.read_csv("elevation_undulation.csv", names=['lat_degree',
                                                        'lat_minute',
                                                        'lat_second',
                                                        'lon_degree',
                                                        'lon_minute',
                                                        'lon_second',
                                                        'geoid',
                                                        'error',
                                                        'ix'], delim_whitespace=True)

print(len(location_undulation))
print(len(elevation_undulation))

389
281


In [18]:
location_joined = location_gcp_decimal.merge(location_undulation, how="left", on="ix")

print(location_joined['ix'].isnull().sum())

df_loc_temp = location_joined[['lat','lon', 'date', 'img_ix', 'elipsoidal_h', 'geoid']]
df_loc_temp['orthoheight'] = df_loc_temp['elipsoidal_h'] - df_loc_temp['geoid']
df_loc = df_loc_temp[['lat','lon', 'orthoheight', 'img_ix', 'date']]


location_orthoheight = gpd.GeoDataFrame(df_loc, geometry=gpd.points_from_xy(df_loc.lon, df_loc.lat))
location_orthoheight = location_orthoheight.set_crs(epsg=4326)
location_orthoheight = location_orthoheight.to_crs("EPSG:32618")
location_orthoheight['lat'] = location_orthoheight.geometry.y
location_orthoheight['lon'] = location_orthoheight.geometry.x
location_ortho_out = pd.DataFrame(location_orthoheight.drop(columns='geometry'))
location_ortho_out.to_csv("location_orthoheight.csv", sep='\t', index=None)

0


<ipython-input-18-d3d98752e5a5>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc_temp['orthoheight'] = df_loc_temp['elipsoidal_h'] - df_loc_temp['geoid']


In [19]:
elevation_joined = elevation_gcp_decimal.merge(elevation_undulation, how="left", on="ix")

print(elevation_joined['ix'].isnull().sum())

df_loc_temp = elevation_joined[['lat','lon', 'date', 'elipsoidal_h', 'geoid']]
df_loc_temp['orthoheight'] = df_loc_temp['elipsoidal_h'] - df_loc_temp['geoid']
df_loc = df_loc_temp[['lat','lon', 'orthoheight', 'date']]


elevation_orthoheight = gpd.GeoDataFrame(df_loc, geometry=gpd.points_from_xy(df_loc.lon, df_loc.lat))
elevation_orthoheight = elevation_orthoheight.set_crs(epsg=4326)
elevation_orthoheight = elevation_orthoheight.to_crs("EPSG:32618")
elevation_orthoheight['lat'] = elevation_orthoheight.geometry.y
elevation_orthoheight['lon'] = elevation_orthoheight.geometry.x
elevation_ortho_out = pd.DataFrame(elevation_orthoheight.drop(columns='geometry'))
elevation_ortho_out.to_csv("elevation_orthoheight.csv", sep='\t', index=None)

0


<ipython-input-19-c18c775fafb0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc_temp['orthoheight'] = df_loc_temp['elipsoidal_h'] - df_loc_temp['geoid']


# parse data for map representation

In [22]:
location_orthoheight.to_file("D:/FEMA project/control_pts/Output_shapefiles/location_gcp.shp")

<ipython-input-22-d7e97b2209f2>:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  location_orthoheight.to_file("D:/FEMA project/control_pts/Output_shapefiles/location_gcp.shp")


In [23]:
elevation_orthoheight.to_file("D:/FEMA project/control_pts/Output_shapefiles/elevation_gcp.shp")

<ipython-input-23-09c35e04fab1>:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  elevation_orthoheight.to_file("D:/FEMA project/control_pts/Output_shapefiles/elevation_gcp.shp")
